In [4]:
!nvidia-smi

Fri Feb  4 01:43:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os 
import json 
import numpy as np 
import pandas as pd 

# Download Dataset

In [7]:
!pip install -q kaggle
!mkdir ~/.kaggle 

In [8]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 5.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=00779224a0ca8f08d40948b53b1fc54635ef181b94d90a545f49439d65b9c720
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [9]:
!rm -rf ~/.kaggle/kaggle.json
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle 
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d nguyentrongquocdat/object-detection-and-drivable-lane-masking

100% 5.81G/5.83G [00:56<00:00, 88.1MB/s]
100% 5.83G/5.83G [00:56<00:00, 111MB/s] 


In [11]:
!unzip /content/object-detection-and-drivable-lane-masking.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/labels/drivable_labels/masks/val/be769e9f-ed37df2b.png  
  inflating: dataset/labels/drivable_labels/masks/val/be76ea62-78b2c8f8.png  
  inflating: dataset/labels/drivable_labels/masks/val/be776742-fe6599b2.png  
  inflating: dataset/labels/drivable_labels/masks/val/be777044-7db298a4.png  
  inflating: dataset/labels/drivable_labels/masks/val/be777044-db29e596.png  
  inflating: dataset/labels/drivable_labels/masks/val/be7783b0-b44ff154.png  
  inflating: dataset/labels/drivable_labels/masks/val/be791a81-a6b53fef.png  
  inflating: dataset/labels/drivable_labels/masks/val/be7b856e-d1ece683.png  
  inflating: dataset/labels/drivable_labels/masks/val/be7c5bf9-f31d60c1.png  
  inflating: dataset/labels/drivable_labels/masks/val/be80a253-75e6f6aa.png  
  inflating: dataset/labels/drivable_labels/masks/val/be819732-9cff62ba.png  
  inflating: dataset/labels/drivable_labels/masks/val/be832d0b-0ed20122.png  
  inflating: 

# Preprocessing Dataset

In [12]:
def create_annotation_file(ignore_small_box_throshold=0):
    for phase in ['train', 'val']: 
        json_data = {'images': [], 'annotations': [], 'categories': []}
        df = pd.read_csv(f'/content/dataset/labels/obj_labels/det_{phase}.csv', index_col='fname')
        img_root = f'/content/dataset/images/{phase}'

        img_map = {}
        for idx, img in enumerate(os.listdir(img_root)):
            img_map[img] = idx
            img_info = {'id': idx, 'width': 1280, 'height': 720, 'file_name': img}
            json_data['images'].append(img_info)

        classes = ['traffic light', 'traffic sign', 'car', 'pedestrian', 'bus', 'truck', 'rider', 'bicycle', 'motorcycle', 'train', 'other vehicle', 'other person', 'trailer']
        label_id = {label: idx for idx, label in enumerate(classes)}
        for idx, label in enumerate(classes):
            category = {'id': idx,
                        'name': label,
                        'supercategory': label}
            json_data['categories'].append(category)

        for idx, (img, row) in enumerate(df.iterrows()):
            x_min, y_min = row['x_min']*1280, row['y_min']*720
            x_max, y_max = row['x_max']*1280, row['y_max']*720
            width, height = x_max-x_min, y_max-y_min
            area = width*height
            if phase == 'train' and area <= ignore_small_box_throshold:
                continue
            annotation = {'id': idx,
                        'image_id': img_map[img],
                        'category_id': label_id[row['category']], 
                        'area': area,
                        'bbox': [x_min, y_min, width, height],
                        'iscrowd': 0}
            json_data['annotations'].append(annotation)
        
        num_images = len(json_data['images'])
        num_objects = len(json_data['annotations'])
        print(f'Phase {phase} contains {num_images} images and {num_objects} objects')
        with open(f'/content/dataset/labels/obj_labels/det_{phase}_coco.json', 'w') as f:
            json.dump(json_data, f) 
        del json_data

create_annotation_file(60)

Phase train contains 70000 images and 1255858 objects
Phase val contains 10000 images and 186033 objects


# Plot Dataset

# MMDetection

In [13]:
!pip install openmim
!mim install mmdet

  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42503 sha256=b553819ec10ce63bf24be5d0e97f87d8ad8965b203f8f921a8754d4bff8407c9
  Stored in directory: /root/.cache/pip/wheels/16/8b/e1/bdebbbc687aa50224a5ce46fe97a040a0c59f92b34bfc750b6
Successfully built openmim
installing mmdet from https://github.com/open-mmlab/mmdetection.git.
Cloning into '/tmp/tmpf0wzc44c/mmdetection'...
remote: Enumerating objects: 22938, done.
remote: Total 22938 (delta 0), reused 0 (delta 0), pack-reused 22938
Receiving objects: 100% (22938/22938), 25.74 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (16074/16074), done.
Note: checking out 'ff9bc39913cb3ff5dde79d3933add7dc2561bab7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or l

In [14]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 22938, done.
remote: Total 22938 (delta 0), reused 0 (delta 0), pack-reused 22938
Receiving objects: 100% (22938/22938), 25.74 MiB | 21.62 MiB/s, done.
Resolving deltas: 100% (16084/16084), done.


## Training 

### Faster-RCNN 

In [15]:
!mkdir /content/mmdetection/checkpoints 

In [16]:
saving_dir = '/content/drive/MyDrive/Project/Autonomous_Vehicle/faster_rcnn_r50_fpn_2x'
os.makedirs(saving_dir, exist_ok=True)

checkpoint_url = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
checkpoint_path = os.path.join(os.getcwd(), checkpoint_url.split('/')[-1])
print(os.path.join('/content/mmdetection/checkpoints', checkpoint_url.split('/')[-1]))
print(checkpoint_url.split('/')[-1])

/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth


In [ ]:
!wget {checkpoint_url}
!mv {checkpoint_path} -t /content/mmdetection/checkpoints 

--2022-02-01 17:06:01--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.72
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167290877 (160M) [application/octet-stream]
Saving to: ‘faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth’

faster_rcnn_r50_fpn 100%[===================>] 159.54M  9.28MB/s    in 17s     

2022-02-01 17:06:20 (9.24 MB/s) - ‘faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth’ saved [167290877/167290877]



In [31]:
content = '''
# The new config inherits a base config to highlight the necessary modification
# {model}_[model setting]_{backbone}_{neck}_[norm setting]_[misc]_[gpu x batch_per_gpu]_{schedule}_{dataset}
_base_ = ['/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_2x_coco.py']

# Change num_classes of model's head 
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=13)))

# Provide pretrained model path to obtain higher performance 
load_from = '/content/latest.pth'

# Optimizer
optimizer = dict(_delete_=True, type='Adam', lr=0.0001)

# Modify log setting  
log_config = dict(
    interval=1000,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')
    ]
)

# Modify dataset related settings
dataset_type = 'COCODataset'
classes = ['traffic light', 'traffic sign', 'car', 
           'pedestrian', 'bus', 'truck', 'rider', 
           'bicycle', 'motorcycle', 'train', 
           'other vehicle', 'other person','trailer']
data = dict(
    samples_per_gpu=8, # Batch size 
    workers_per_gpu=8, # Number of workers
    train=dict(
        img_prefix='/content/dataset/images/train',
        classes=classes,
        ann_file='/content/dataset/labels/obj_labels/det_train_coco.json'),
    val=dict(
        img_prefix='/content/dataset/images/val',
        classes=classes,
        ann_file='/content/dataset/labels/obj_labels/det_val_coco.json'),
    test=dict(
        img_prefix='/content/dataset/images/val',
        classes=classes,
        ann_file='/content/dataset/labels/obj_labels/det_val_coco.json'))
'''

with open('config.py', 'w') as f:
    f.write(content)

In [29]:
!python /content/mmdetection/tools/misc/print_config.py /content/config.py 

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

In [ ]:
!python /content/mmdetection/tools/train.py /content/config.py \
                                            --work-dir {saving_dir} \

fatal: not a git repository (or any of the parent directories): .git
2022-02-01 17:07:11,646 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.10.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genco

In [17]:
last_checkpoint = os.path.join(saving_dir, 'latest.pth')
last_config = os.path.join(saving_dir, 'config.py')

In [19]:
!python /content/mmdetection/tools/train.py {last_config} \
                                            --auto-resume \
                                            --work-dir {saving_dir}

fatal: not a git repository (or any of the parent directories): .git
2022-02-04 01:55:28,423 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Jan 15 2022, 18:48:18) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.10.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genco

In [33]:
!python /content/mmdetection/tools/train.py /content/config.py \
                                            --work-dir /content/drive/MyDrive/Project/Autonomous_Vehicle/faster_rcnn_r50_fpn_2x_Adam

fatal: not a git repository (or any of the parent directories): .git
2022-02-04 04:59:24,037 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Jan 15 2022, 18:48:18) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.10.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genco

In [ ]:
config_path = os.path.join(saving_dir, 'config_py')
eval_checkpoint = os.path.join(saving_dir, 'latest.pth')

!python /content/mmdetection/tools/test.py {config_path} \
                                           {eval_checkpoint} \
                                           --eval bbox

loading annotations into memory...
Done (t=1.09s)
creating index...
index created!
load checkpoint from local path: /content/work_dirs/config/latest.pth
[>>] 10000/10000, 13.1 task/s, elapsed: 763s, ETA:     0sTraceback (most recent call last):
  File "/content/mmdetection/tools/test.py", line 251, in <module>
    main()
  File "/content/mmdetection/tools/test.py", line 243, in main
    metric = dataset.evaluate(outputs, **eval_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/mmdet/datasets/coco.py", line 408, in evaluate
    raise KeyError(f'metric {metric} is not supported')
KeyError: 'metric mAP is not supported'
